In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [18]:
cd C:/Users/0105l/Desktop/데이터/

C:\Users\0105l\Desktop\데이터


In [20]:
input = torch.randn(100,3,3) #(batch, seq_len_, input_size)
input_size= 3
seq_length = 3
hidden_size = 6
num_layers = 2
dropout_rate= 0.5
num_classes = 2

In [21]:
def distance(x1, y1, x2, y2):
    m = (y2-y1) / (x2-x1)
    return m

In [143]:
# 포지션 => 기울기로 변환
# coef.shape = (300,1)
def Coefficient(df): 
    coef = []
    for i in range(49, 351):
        frame1 = df[i][gradlab1][0:2]
        frame2 = df[i][gradlab2][0:2]
        coef.append(distance(frame1[0], frame1[1], frame2[0], frame2[1]))
    coef = pd.DataFrame(coef, columns= ['coef'])
    
    q3=coef['coef'].quantile(0.75)
    q1=coef['coef'].quantile(0.25)
    iqr=q3-q1
    outlier_top=q3+1.5*iqr
    outlier_bottom=q1-1.5*iqr
    idx=coef[(coef['coef']<outlier_bottom) | (coef['coef']>outlier_top)].index
    
    idx_list=[]
    for i in idx:
        idx_list.append(str(i))
    idx_list.append('end')

    idx_string = ' '.join(idx_list)
    idx_string_split = idx_string.split('end')
    outlier_str = idx_string_split[0].strip()
    outlier = list(map(int, outlier_str.split(' ')))

    for i in range(len(coef)-1):
        if i in outlier:
            if i == 0:
                continue
            elif i != (len(coef)-1):
                coef['coef'][i] = (coef['coef'][i-1] + coef['coef'][i+1])/2
            else:
                break
                
    coef = np.array(coef)[1:301]
    
    return coef

In [144]:
# 가속도 넘파이 이상치 변환
# acc_array.shape = (300,2)
def Acceleration(pos_1, array_1, acclab):
    df = pd.DataFrame(array_1[:,acclab,:], columns=['x','y','z'])
    idx_list=[]
    for i in ['x','y','z']:
        q3=df[i].quantile(0.75)
        q1=df[i].quantile(0.25)
        iqr=q3-q1
        outlier_top=q3+1.5*iqr
        outlier_bottom=q1-1.5*iqr

        idx=df[(df[i]<outlier_bottom) | (df[i]>outlier_top)].index
        for i in idx:
            idx_list.append(str(i))
        idx_list.append('end')
    
    idx_string = ' '.join(idx_list)
    idx_string_split = idx_string.split('end')
    x_outlier_str = idx_string_split[0].strip()
    y_outlier_str = idx_string_split[1].strip()
    x_outlier = list(map(int, x_outlier_str.split(' ')[:-1]))
    y_outlier = list(map(int, y_outlier_str.split(' ')[:-1]))
    
    acc_1_parsed = array_1[:,acclab,:]
    for i in range(len(pos_1)-1):
        if i in x_outlier:
            acc_1_parsed[i][0] = (acc_1_parsed[i-1][0] + acc_1_parsed[i+1][0])/2 
        if i in y_outlier:
            acc_1_parsed[i][1] = (acc_1_parsed[i-1][1] + acc_1_parsed[i+1][1])/2 
       
    final_df = pd.DataFrame(acc_1_parsed, columns = ['x','y','z'])
    acc_array = StandardScaler().fit_transform(final_df)
    acc_array = acc_array[49:349,:]
    
    return acc_array[:,:2] 

In [145]:
#부위 별로 모델을 만들거라 상관없다 ..? (오답 데이터 가져오는 거 생각할 필요 있음) 
hand = 'Right'
gesture_list = ['URF','UMF','ULF','DLF','DMF','DRF','URB','UMB','ULB','DLB','DMB','DRB','URO','ULO','DLO','DRO']
right_gesture = gesture_list[0]
num_session = 5
#총 세션 개수 = batch size
#5로 적어야 1~4까지 들어감

acclab1 = 8
acclab2 = 12
gradlab1 = 8
gradlab2 = 12


In [146]:
def dataloader(gesture):
    data2 = np.zeros((1,300,6))
    for i in range(1, num_session):
        data = np.zeros((300,1))
        pos = np.load('%s_%s_p_%d.npy' % (hand, gesture, i))
        acc = np.load('%s_%s_a_%d.npy' % (hand, gesture, i))
        acc1 = Acceleration(pos, acc, acclab1)
        acc2 = Acceleration(pos, acc, acclab2)
        coef = Coefficient(pos)
        if gesture == right_gesture:
            label = np.ones((300,1))
        else:
            label = np.zeros((300,1))
        data = np.hstack((data,acc1,acc2,coef,label))
        data = data[:, 1:].reshape((1,300,6))
        data2 = np.vstack((data2,data))
    
        
    return data2

In [149]:
target_data = dataloader('URF')
error_data = dataloader('DLB')

In [150]:
target_data.shape, error_data.shape

((5, 300, 6), (5, 300, 6))

In [67]:
acc = np.load('Left_ULF_a_5.npy')
acc.shape

(398, 21, 3)

In [47]:
class LSTM_Chicka(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length, dropout_rate):
        super(LSTM_Chicka,self).__init__()
        self.lstm =  nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout_rate, batch_first = True)
        self.layer_1 = nn.Linear(hidden_size, 256)
        self.layer_2 = nn.Linear(256,64)
        self.layer_3 = nn.Linear(64,16)
        self.layer_out = nn.Linear(16, num_classes) 
        self.relu = nn.ReLU()  
        
    def forward(self,x):
        h_0 = torch.tensor(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) # x.size(0) batch size 
        c_0 = torch.tensor(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        output, (hn,cn) = self.lstm(x,(h_0,c_0))
        
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.layer_1(out) 
        out = self.relu(out) 
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.relu(out)
        out = self.layer_out(out) 

        return out

In [46]:
model = LSTM_Chicka(num_classes, input_size, hidden_size, num_layers, seq_length, dropout_rate)
criterion = torch.nn.BCELoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer= optimizer)